In [1]:
import numpy as np
import pandas as pd
import skyfield
from skyfield.api import load
from skyfield.positionlib import ICRF, Barycentric
import astropy
from astropy.units import deg, au, km, meter, day, minute, second
from astropy.coordinates import SkyCoord, ICRS, GCRS, BarycentricMeanEcliptic, HeliocentricMeanEcliptic, EarthLocation
from scipy.interpolate import CubicSpline
import matplotlib.pyplot as plt

# MSE imports
import astro_utils
from astro_utils import jd_to_mjd
from ra_dec import radec2dir, qv2dir, dir2radec, radec_diff
import asteroid_integrate
from asteroid_data import make_data_one_file, get_earth_pos

### Observation of Earth and Mars according to JPL

In [66]:
def add_col_planet(df):
    """Add a new column to a DataFrame with the MJD"""
    # Add column for mjd
    df['mjd'] = (df['JulianDate'] - 2400000.5)
    # integer column for the date/time
    df['time_key'] = np.int32(np.round(df['mjd']*24))    
    # Order columns
    columns = ['mjd', 'JulianDate', 'time_key', 'X', 'Y', 'Z', 'VX', 'VY', 'VZ', 'LT', 'RG', 'RR']
    df = df[columns]
    return df

In [67]:
# Load the JPL position of Earth as CSV
df_earth = pd.read_csv('../data/jpl/testing/daily/vectors-earth.txt', index_col=False)
# Add column for mjd
df_earth = add_col_planet(df_earth)

# Display the dataframe
df_earth

,mjd,JulianDate,time_key,X,Y,Z,VX,VY,VZ,LT,RG,RR
0,55197.0,2455197.5,1324728,-0.179765,0.970347,-0.000017,-0.017202,-0.003148,8.961125e-07,0.005700,0.986858,0.000038
1,55198.0,2455198.5,1324752,-0.196939,0.967049,-0.000017,-0.017145,-0.003447,9.036109e-07,0.005700,0.986899,0.000044
2,55199.0,2455199.5,1324776,-0.214053,0.963453,-0.000016,-0.017083,-0.003745,8.653246e-07,0.005700,0.986945,0.000049
3,55200.0,2455200.5,1324800,-0.231103,0.959559,-0.000015,-0.017017,-0.004042,7.855759e-07,0.005700,0.986997,0.000054
4,55201.0,2455201.5,1324824,-0.248085,0.955369,-0.000014,-0.016945,-0.004339,6.725245e-07,0.005701,0.987054,0.000059
...,...,...,...,...,...,...,...,...,...,...,...,...
3648,58845.0,2458845.5,1412280,-0.100787,0.986067,-0.000022,-0.017416,-0.001767,9.809367e-07,0.005725,0.991204,0.000013
3649,58846.0,2458846.5,1412304,-0.118187,0.984147,-0.000021,-0.017382,-0.002073,9.327289e-07,0.005725,0.991218,0.000015
3650,58847.0,2458847.5,1412328,-0.135550,0.981922,-0.000020,-0.017343,-0.002377,8.590659e-07,0.005725,0.991234,0.000017
3651,58848.0,2458848.5,1412352,-0.152871,0.979393,-0.000019,-0.017298,-0.002681,7.650397e-07,0.005725,0.991251,0.000019


In [68]:
# Load the JPL position of Mars as CSV
df_mars = pd.read_csv('../data/jpl/testing/daily/vectors-mars.txt', index_col=False)
# Add column for mjd
df_mars = add_col_planet(df_mars)

# Display the dataframe
df_mars

,mjd,JulianDate,time_key,X,Y,Z,VX,VY,VZ,LT,RG,RR
0,55197.0,2455197.5,1324728,-0.733418,1.457212,0.048394,-0.011980,-0.005093,0.000188,0.009426,1.632088,0.000842
1,55198.0,2455198.5,1324752,-0.745374,1.452070,0.048580,-0.011930,-0.005192,0.000184,0.009431,1.632927,0.000834
2,55199.0,2455199.5,1324776,-0.757278,1.446828,0.048763,-0.011879,-0.005291,0.000181,0.009436,1.633756,0.000826
3,55200.0,2455200.5,1324800,-0.769131,1.441488,0.048942,-0.011827,-0.005390,0.000178,0.009441,1.634578,0.000817
4,55201.0,2455201.5,1324824,-0.780931,1.436049,0.049118,-0.011774,-0.005488,0.000174,0.009445,1.635391,0.000809
...,...,...,...,...,...,...,...,...,...,...,...,...
3648,58845.0,2458845.5,1412280,-1.356374,-0.836106,0.015533,0.007920,-0.010678,-0.000418,0.009203,1.593445,-0.001143
3649,58846.0,2458846.5,1412304,-1.348404,-0.846754,0.015114,0.008019,-0.010616,-0.000419,0.009196,1.592299,-0.001149
3650,58847.0,2458847.5,1412328,-1.340336,-0.857339,0.014695,0.008117,-0.010553,-0.000420,0.009190,1.591146,-0.001155
3651,58848.0,2458848.5,1412352,-1.332170,-0.867860,0.014274,0.008215,-0.010490,-0.000421,0.009183,1.589988,-0.001161


In [69]:
def add_col_obs(df):
    """Add a new column to a DataFrame with the MJD"""
    # Add column for mjd
    df['mjd'] = (df['JulianDate'] - 2400000.5)
    # integer column for the date/time
    df['time_key'] = np.int32(np.round(df['mjd']*24))
    # Add columns for the direction in ecliptic
    u = radec2dir(ra=df.RA.values * deg, dec=df.DEC.values * deg, obstime_mjd = df['mjd'].values)
    df['u_x'] = u[0]
    df['u_y'] = u[1]
    df['u_z'] = u[2]
    # Order columns
    columns = ['mjd', 'JulianDate', 'time_key', 'u_x', 'u_y', 'u_z', 'RA', 'DEC', 
               'RA_apparent', 'DEC_apparent',
               'delta', 'delta_dot', 'light_time',]
    df = df[columns]
    return df

In [70]:
# Load the JPL RA and DEC for Mars as fixed width file
df_obs_mars = pd.read_fwf('../data/jpl/testing/daily/observe-mars-geocenter.txt')
# Add column for mjd
df_obs_mars = add_col_obs(df_obs_mars)

# Display the dataframe
df_obs_mars

,mjd,JulianDate,time_key,u_x,u_y,u_z,RA,DEC,RA_apparent,DEC_apparent,delta,delta_dot,light_time
0,55197.0,2455197.5,1324728,-0.749289,0.658994,0.065524,142.327061,18.799029,142.475968,18.752304,0.738832,-8.970408,6.144676
1,55198.0,2455198.5,1324752,-0.747395,0.661070,0.066233,142.179197,18.888379,142.328371,18.841711,0.733722,-8.724067,6.102178
2,55199.0,2455199.5,1324776,-0.745339,0.663317,0.066933,142.017468,18.981461,142.166897,18.934867,0.728756,-8.470333,6.060882
3,55200.0,2455200.5,1324800,-0.743120,0.665733,0.067625,141.841890,19.078173,141.991560,19.031671,0.723940,-8.208951,6.020822
4,55201.0,2455201.5,1324824,-0.740736,0.668315,0.068307,141.652504,19.178403,141.802402,19.132004,0.719276,-7.939695,5.982036
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3648,58845.0,2458845.5,1412280,-0.567439,-0.823385,0.007031,233.190100,-18.727810,233.467326,18.791714,2.212868,12.191696,8.403866
3649,58846.0,2458846.5,1412304,-0.557754,-0.829978,0.006864,233.876730,-18.895847,234.154585,18.958738,2.205809,12.255895,8.345152
3650,58847.0,2458847.5,1412328,-0.547988,-0.836459,0.006694,234.565048,-19.061459,234.843518,19.123323,2.198712,12.318649,8.286133
3651,58848.0,2458848.5,1412352,-0.538142,-0.842829,0.006524,235.255049,-19.224614,235.534120,19.285436,2.191580,12.379967,8.226816


In [71]:
# Extract position and velocity of earth from df_earth
q_earth_jpl = np.array([df_earth.X.values, df_earth.Y.values, df_earth.Z.values]) * au
v_earth_jpl = np.array([df_earth.VX.values, df_earth.VY.values, df_earth.VZ.values]) * au / day

# Extract position of mars from df_mars
q_mars_jpl = np.array([df_mars.X.values, df_mars.Y.values, df_mars.Z.values]) * au
v_mars_jpl = np.array([df_mars.VX.values, df_mars.VY.values, df_mars.VZ.values]) * au / day

# Extract obstime_jd, ra, and dec from DataFrame
obstime_mars_jd = df_obs_mars.JulianDate.values
ra_mars_jpl = df_obs_mars.RA.values * deg
dec_mars_jpl = df_obs_mars.DEC.values * deg

# Vector of observation times in MJD format
obstime_mars_mjd = jd_to_mjd(obstime_mars_jd)

### Position & Observation of Earth and Mars according to Skyfield

In [72]:
# Manually load planetary positions using de435
planets_sf = load('../data/jpl/ephemeris/de435.bsp')
earth_sf = planets_sf['earth']
mars_sf = planets_sf['mars barycenter']

# load timescale
ts_sf = load.timescale()

# Generate vector of observation times in Skyfield format
obstime_mars_sf = ts_sf.tt_jd(obstime_mars_jd)

In [73]:
# Observe mars from earth with Skyfield
obs_mars_sf = earth_sf.at(obstime_mars_sf).observe(mars_sf)

# Build Skyfield angle arrays (RA, DEC) and distance array (delta)
ra_mars_sf_aa, dec_mars_sf_aa, delta_mars_sf_da = obs_mars_sf.radec()

# Extract degrees and AU to get arrays of astropy angles and distances
ra_mars_sf = ra_mars_sf_aa._degrees * deg
dec_mars_sf = dec_mars_sf_aa._degrees * deg
delta_mars_sf = delta_mars_sf_da.au * au

In [74]:
# Load planetary positions and velocities by querying the Skyfield JPL ephemeris interface
# Create them as arrays with bundled astropy units of au and km / second

# Earth
q_earth_sf = earth_sf.at(obstime_mars_sf).ecliptic_position().au * au
v_earth_sf = earth_sf.at(obstime_mars_sf).ecliptic_velocity().km_per_s * km / second

# Mars
q_mars_sf = mars_sf.at(obstime_mars_sf).ecliptic_position().au * au
v_mars_sf = mars_sf.at(obstime_mars_sf).ecliptic_velocity().km_per_s * km / second

In [75]:
# Demonstrate that q_earth_sf is the same as q_earth_jpl
q_earth_eps = np.mean(np.linalg.norm(q_earth_sf - q_earth_jpl, axis=0))
v_earth_eps = np.mean(np.linalg.norm(v_earth_sf - v_earth_jpl, axis=0))
q_mars_eps = np.mean(np.linalg.norm(q_mars_sf - q_mars_jpl, axis=0))
v_mars_eps = np.mean(np.linalg.norm(v_mars_sf - v_mars_jpl, axis=0))

# Report
print('Difference between Skyfield (JPL ephem) and Horizons download:')
print(f'q_earth : {q_earth_eps:5.3e} au')
print(f'v_earth : {v_earth_eps:5.3e} au / day')
print(f'q_mars  : {q_mars_eps:5.3e} au')
print(f'v_mars  : {v_mars_eps:5.3e} au / day')

Difference between Skyfield (JPL ephem) and Horizons download:
q_earth : 1.406e-09 au
v_earth : 1.626e-08 au / day
q_mars  : 1.565e-09 au
v_mars  : 3.758e-08 au / day


### Compare Skyfield vs JPL on RA/DEC of Mars

In [76]:
# Compute difference in angles
diff_sf = radec_diff('JPL', 'Skyfield', ra1=ra_mars_jpl, dec1=dec_mars_jpl, ra2=ra_mars_sf, dec2=dec_mars_sf, 
                     obstime_mjd=obstime_mars_mjd, verbose=True)

Angle Difference: Skyfield vs. JPL
Mean  :   0.000444 deg (   1.598 seconds)
Median:   0.000506 deg (   1.822 seconds)
Max   :   0.000615 deg (   2.213 seconds)


In [77]:
# Compute direction from Earth to Mars using the RA/DEC from JPL
u_mars_jpl = radec2dir(ra_mars_jpl, dec_mars_jpl, obstime_mars_mjd)

# Compute direction from Earth to Mars using the RA/DEC from Skyfield
u_mars_sf = radec2dir(ra_mars_sf, dec_mars_sf, obstime_mars_mjd)

# Difference in directions
u_mars_diff = u_mars_sf - u_mars_jpl

# Norm of difference
u_diff_norm = np.linalg.norm(u_mars_diff, axis=0)

# Mean
u_diff_mean = np.mean(u_diff_norm)
print(f'Mean norm of direction difference: JPL vs. Skyfield')
print(f'Mean : {u_diff_mean:5.2e}')

Mean norm of direction difference: JPL vs. Skyfield
Mean : 7.75e-06


### Calculate Direction Earth to Mars with qv2dir() and JPL Position / Velocity

In [78]:
# Calculate direction of Mars with JPL position data
u_mars_mse = qv2dir(q_body=q_mars_jpl, v_body=v_mars_jpl, q_earth=q_earth_jpl)

In [79]:
# difference in directions as a vector
u_diff = u_mars_mse - u_mars_jpl

# norm of difference, converted to arc seconds
u_diff_norm = np.linalg.norm(u_diff, axis=0)
angle_diff = np.rad2deg(u_diff_norm)*3600

# mean error in arc-seconds
mean_error = np.mean(angle_diff)
print(f'Observation of Mars: MSE qv2dir() on JPL position vs. JPL direction')
print(f'mean error: {mean_error:8.3f} arc seconds')

Observation of Mars: MSE qv2dir() on JPL position vs. JPL direction
mean error:    1.604 arc seconds


In [80]:
# difference in directions as a vector
u_diff = u_mars_mse - u_mars_sf

# norm of difference, converted to arc seconds
u_diff_norm = np.linalg.norm(u_diff, axis=0)
angle_diff = np.rad2deg(u_diff_norm)*3600

# mean error in arc-seconds
mean_error = np.mean(angle_diff)
print(f'Observation of Mars: MSE qv2dir() on JPL position vs. Skyfield direction')
print(f'mean error: {mean_error:8.3f} arc seconds')

Observation of Mars: MSE qv2dir() on JPL position vs. Skyfield direction
mean error:    0.027 arc seconds


**Conclusion**<br>
My calculations are almost identical to Skyfield; accurate to **0.027 arc seconds**<br>
Both Skyfield and I are off from JPL by **1.60 arc seconds**<br>

### Calculate RA/DEC of Mars with MSE Function dir2radec() and JPL Directions

In [81]:
ra_mars_mse, dec_mars_mse = dir2radec(u_mars_jpl, obstime_mjd=obstime_mars_mjd)

In [82]:
# Compute difference in angles
diff_mse = radec_diff('JPL', 'MSE', ra1=ra_mars_jpl, dec1=dec_mars_jpl, ra2=ra_mars_sf, dec2=dec_mars_sf, 
                     obstime_mjd=obstime_mars_mjd, verbose=True)

Angle Difference: MSE vs. JPL
Mean  :   0.000444 deg (   1.598 seconds)
Median:   0.000506 deg (   1.822 seconds)
Max   :   0.000615 deg (   2.213 seconds)


### Vectors of First 16 Asteroids from JPL

In [83]:
def add_cols_ast_vector(df, asteroid_num: int):
    """Add columns to a DataFrame with asteroid vectors; new columns for asteroid number and mjd"""
    # Add column for asteroid_num
    df['asteroid_num'] = asteroid_num
    # Add column for mjd
    df['mjd'] = (df['JulianDate'] - 2400000.5)
    # Add integer column for the date/time
    df['time_key'] = np.int32(np.round(df['mjd']*24))
    # Order columns
    columns = ['asteroid_num', 'mjd', 'JulianDate', 'time_key', 'X', 'Y', 'Z', 'VX', 'VY', 'VZ', 'LT', 'RG', 'RR']
    df = df[columns]
    return df

In [84]:
# List of dataframes; one per asteroid
df_ast_list = []

# Load the JPL position of asteroids one at a time
for j in range(1, 17):
    # print(j)
    df_ast_j = pd.read_csv(f'../data/jpl/testing/daily/vectors-asteroid-{j:03d}.txt', index_col=False)
    # Add columns for the asteroid_num and mjd
    df_ast_j = add_cols_ast_vector(df=df_ast_j, asteroid_num=j)
    # Add this to list of frames
    df_ast_list.append(df_ast_j)

# Concatenate dataframes
df_ast = pd.concat(df_ast_list)

In [85]:
 df_ast

,asteroid_num,mjd,JulianDate,time_key,X,Y,Z,VX,VY,VZ,LT,RG,RR
0,1,55197.0,2455197.5,1324728,-1.660333,-2.123236,0.238962,0.007615,-0.007150,-0.001627,0.015628,2.705909,0.000794
1,1,55198.0,2455198.5,1324752,-1.652706,-2.130370,0.237334,0.007640,-0.007118,-0.001630,0.015633,2.706703,0.000794
2,1,55199.0,2455199.5,1324776,-1.645053,-2.137472,0.235702,0.007665,-0.007086,-0.001634,0.015637,2.707497,0.000795
3,1,55200.0,2455200.5,1324800,-1.637376,-2.144542,0.234066,0.007689,-0.007054,-0.001637,0.015642,2.708293,0.000796
4,1,55201.0,2455201.5,1324824,-1.629675,-2.151580,0.232427,0.007713,-0.007022,-0.001641,0.015646,2.709088,0.000796
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3648,16,58845.0,2458845.5,1412280,2.517677,-0.513079,-0.043698,0.001590,0.011210,-0.000568,0.014842,2.569797,-0.000671
3649,16,58846.0,2458846.5,1412304,2.519245,-0.501865,-0.044266,0.001546,0.011219,-0.000568,0.014838,2.569129,-0.000665
3650,16,58847.0,2458847.5,1412328,2.520770,-0.490641,-0.044834,0.001503,0.011228,-0.000567,0.014834,2.568466,-0.000660
3651,16,58848.0,2458848.5,1412352,2.522251,-0.479409,-0.045400,0.001459,0.011236,-0.000566,0.014830,2.567809,-0.000655


### Observations of First 16 Asteroids from JPL

In [86]:
def add_cols_ast_observation(df, asteroid_num: int):
    """Add columns to a DataFrame with asteroid vectors; new columns for asteroid number, mjd, and direction"""
    # Add column for asteroid_num
    df['asteroid_num'] = asteroid_num
    # Add column for mjd
    df['mjd'] = (df['JulianDate'] - 2400000.5)
    # Add integer column for the date/time
    df['time_key'] = np.int32(np.round(df['mjd']*24))
    # Add columns for the direction in ecliptic
    u = radec2dir(ra=df.RA.values * deg, dec=df.DEC.values * deg, obstime_mjd = df['mjd'].values)
    df['u_x'] = u[0]
    df['u_y'] = u[1]
    df['u_z'] = u[2]
    # Order columns
    columns = ['asteroid_num', 'mjd', 'JulianDate', 'time_key',
               'u_x', 'u_y', 'u_z', 'RA', 'DEC', 
               'RA_apparent', 'DEC_apparent',
               'delta', 'delta_dot', 'light_time',]
    df = df[columns]
    return df

In [87]:
# List of dataframes; one per asteroid
df_obs_list = []

# Load the JPL observations of asteroids one at a time
for j in range(1, 17):
    # print(j)
    df_obs_j = pd.read_fwf(f'../data/jpl/testing/daily/observer-asteroid-{j:03d}-geocenter.txt', index_col=False)
    # Add columns for the asteroid_num and mjd
    df_obs_j = add_cols_ast_observation(df=df_obs_j, asteroid_num=j)
    # Add this to list of frames
    df_obs_list.append(df_obs_j)

# Concatenate dataframes
df_obs = pd.concat(df_obs_list)

In [88]:
df_obs

,asteroid_num,mjd,JulianDate,time_key,u_x,u_y,u_z,RA,DEC,RA_apparent,DEC_apparent,delta,delta_dot,light_time
0,1,55197.0,2455197.5,1324728,-0.430702,-0.899812,0.069523,243.215442,-17.105913,243.358581,-17.131844,3.437877,-12.468091,28.591952
1,1,55198.0,2455198.5,1324752,-0.424384,-0.902835,0.069195,243.625145,-17.196033,243.768548,-17.221645,3.430618,-12.668422,28.531584
2,1,55199.0,2455199.5,1324776,-0.418063,-0.905804,0.068867,244.034084,-17.284935,244.177730,-17.310227,3.423244,-12.868277,28.470254
3,1,55200.0,2455200.5,1324800,-0.411741,-0.908720,0.068539,244.442231,-17.372621,244.586099,-17.397589,3.415755,-13.067723,28.407966
4,1,55201.0,2455201.5,1324824,-0.405417,-0.911583,0.068211,244.849560,-17.459094,244.993632,-17.483728,3.408150,-13.266749,28.344720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3648,16,58845.0,2458845.5,1412280,0.867713,-0.496854,-0.014471,332.583400,-12.175911,332.842722,-12.079671,3.017646,17.406609,25.096998
3649,16,58846.0,2458846.5,1412304,0.871109,-0.490873,-0.014611,332.963279,-12.044021,333.222363,-11.947459,3.027660,17.268707,25.180280
3650,16,58847.0,2458847.5,1412328,0.874478,-0.484841,-0.014750,333.344527,-11.910963,333.603364,-11.814086,3.037593,17.128990,25.262894
3651,16,58848.0,2458848.5,1412352,0.877821,-0.478758,-0.014889,333.727108,-11.776749,333.985688,-11.679564,3.047446,16.987592,25.344833


### Calculate Asteroid Direction with qv2dir() and JPL Position / Velocity

In [89]:
# arrays of asteroid position & velocity from JPL
q_ast_jpl = np.array([df_ast.X.values, df_ast.Y.values, df_ast.Z.values]) * au
v_ast_jpl = np.array([df_ast.VX.values, df_ast.VY.values, df_ast.VZ.values]) * au / day

# tile earth position to match shape of q_ast
q_earth_jpl_tile = np.tile(q_earth_jpl, 16)

# arrays of asteroid angles from JPL
ra_jpl = df_obs.RA.values * deg
dec_jpl = df_obs.DEC.values * deg
obstime_mjd = df_obs.mjd.values.astype(np.float64)

# the direction according to JPL; convert RA/DEC to direction in ecliptic plane
u_jpl = radec2dir(ra=ra_jpl, dec=dec_jpl, obstime_mjd=obstime_mjd)

# calculate direction with JPL data
u_mse = qv2dir(q_body=q_ast_jpl, v_body=v_ast_jpl, q_earth=q_earth_jpl_tile)

# difference in directions as a vector
u_diff = u_mse - u_jpl

# norm of difference, converted to arc seconds
u_diff_norm = np.linalg.norm(u_diff, axis=0)
angle_diff = np.rad2deg(u_diff_norm)*3600

# mean error in arc-seconds
mean_error = np.mean(angle_diff)
print(f'Observation of Asteroids: MSE qv2dir() on JPL position vs. JPL direction')
print(f'mean error: {mean_error:8.3f} arc seconds')

Observation of Asteroids: MSE qv2dir() on JPL position vs. JPL direction
mean error:    0.873 arc seconds


**Conclusion:<br>**
qv2dir() is highly accurate in computing a right ascension and declination from position and velocity in the barycentric ecliptic plane.<br>
Errors are on the order of **0.87 arc seconds**.<br>
Differences with JPL are due to using a linear approximation to the adjustment of the space body's position due to light lag.  The JPL calculation is iteratively solving for the position of the body on its true orbit at the instant photons leaving it hit the earth at print time.  This simplified calculation is applying an adjustment of the form<br>
```
r = norm(q_body - q_earth)
light_time = r / light_speed
dq = v_body * light_time
```

In [90]:
# Compute RA and DEC from direction computed by radec2dir() on the JPL data
ra_mse, dec_mse = dir2radec(u_jpl, obstime_mjd)

In [91]:
# Compute difference in angles
diff_mse = radec_diff('JPL', 'MSE', ra1=ra_jpl, dec1=dec_jpl, ra2=ra_mse, dec2=dec_mse, 
                     obstime_mjd=obstime_mjd, verbose=False)
diff_mean = np.mean(diff_mse)
diff_median = np.median(diff_mse)
diff_max = np.max(diff_mse)

# Report results
print(f'Mean Angle Difference: JPL vs. MSE from JPL positions')
print(f'Mean  : {diff_mean:5.3e} seconds')
print(f'Median: {diff_median:5.3e} seconds')
print(f'Max   : {diff_max:5.3e} seconds')

Mean Angle Difference: JPL vs. MSE from JPL positions
Mean  : 5.826e-11 seconds
Median: 1.717e-11 seconds
Max   : 4.032e-10 seconds


**Conclusion:<br>**
The round trip of radec2dir() and dir2radec() is accurate on the order of double precision.<br>
In the test, a direction was computed from the RA and DEC provided by JPL. This was then converted back to a RA and DEC.<br>
Errors are on the order of **5.8E-11 arc seconds**.<br>

### Calculate Asteroid Direction with qv2dir() and MSE Position / Velocity

In [92]:
ast_elt = asteroid_integrate.load_data()
ast_elt.rename(mapper={'Num':'asteroid_num'}, axis='columns', inplace=True)

In [93]:
# Range of asteroids to for data
ast_num_file_start: int = 1
ast_num_file_end: int = 1000
inputs, outputs = make_data_one_file(0, ast_num_file_end)

In [94]:
ast_elt

,asteroid_num,Name,epoch_mjd,a,e,inc,Omega,omega,M,H,G,Ref,f,P,n,long,theta,pomega,T_peri
Num,,,,,,,,,,,,,,,,,,,
1,1,Ceres,58600.0,2.769165,0.076009,0.184901,1.401596,1.284522,1.350398,3.34,0.12,JPL 46,1.501306,1683.145749,0.003733,4.036516,4.187424,2.686118,-361.745873
2,2,Pallas,58600.0,2.772466,0.230337,0.608007,3.020817,5.411373,1.041946,4.13,0.11,JPL 35,1.490912,1686.155979,0.003726,3.190951,3.639917,2.149005,-279.616804
3,3,Juno,58600.0,2.669150,0.256942,0.226699,2.964490,4.330836,0.609557,5.33,0.32,JPL 108,0.996719,1592.787270,0.003945,1.621697,2.008860,1.012141,-154.522558
4,4,Vesta,58600.0,2.361418,0.088721,0.124647,1.811840,2.630709,1.673106,3.20,0.32,JPL 34,-4.436417,1325.432768,0.004740,6.115656,0.006132,4.442550,-352.940421
5,5,Astraea,58600.0,2.574249,0.191095,0.093672,2.470978,6.260280,4.928221,6.85,0.15,JPL 108,-1.738676,1508.600442,0.004165,1.093108,0.709396,2.448072,325.328481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541124,541124,2018 RP23,58600.0,2.586399,0.289358,0.088749,2.000720,3.913328,1.075531,17.30,0.15,JPL 7,1.654537,1519.293350,0.004136,0.706394,1.285400,-0.369137,-260.066715
541125,541125,2018 RV23,58600.0,3.113036,0.213678,0.203046,0.544794,0.242079,0.130760,16.10,0.15,JPL 8,0.206083,2006.201725,0.003132,0.917633,0.992956,0.786873,-41.751113
541126,541126,2018 RP24,58600.0,2.453880,0.176693,0.194504,2.649626,3.695880,0.937231,17.30,0.15,JPL 6,1.258854,1404.036362,0.004475,0.999551,1.321174,0.062320,-209.433076


In [95]:
# The block of asteroid numbers to test (inclusive boundaries)
ast_num_min = 1
ast_num_max = 16

# The number of asteroids, times, and total rows we want to match
N_ast = ast_num_max - ast_num_min + 1
N_t = df_earth.mjd.size
N_row = N_ast * N_t

# Report data shape
print(f'Shape of data frames df_ast and df_obs:')
print(f'N_ast = {N_ast:5} asteroids')
print(f'N_t   = {N_t:5} observation times')
print(f'N_row = {N_row:5} rows in df_ast and df_obs')

Shape of data frames df_ast and df_obs:
N_ast =    16 asteroids
N_t   =  3653 observation times
N_row = 58448 rows in df_ast and df_obs


In [96]:
# Filter for asteroid numbers 
ast_num_file = np.arange(ast_num_file_start, ast_num_file_end, dtype=np.int64)
mask_ast = (ast_num_min <= ast_num_file) & (ast_num_file <= ast_num_max)

# MSE integrated times as one array
ts = inputs['ts'][0]

# Time range for JPL data
t_min = np.min(obstime_mjd)
t_max = np.max(obstime_mjd)

# Filter for MSE times that match
mask_t = (t_min <= ts) & (ts <= t_max)

In [97]:
# Block of asteroid data 
q_ast_all = outputs['q']
v_ast_all = outputs['v']

# filter for selected asteroids only
q_ast_all_t = q_ast_all[mask_ast, :, :]
v_ast_all_t = v_ast_all[mask_ast, :, :]

# filter for selected times only
q_ast_mse_3d = q_ast_all_t[:, mask_t, :]
v_ast_mse_3d = v_ast_all_t[:, mask_t, :]

# for some reason i don't understand, can't do these at once
# q_ast_mse = q_ast_all[mask_ast, mask_t, :]
q_ast_mse_3d.shape

(16, 3653, 3)

In [98]:
# Get position of Earth using utility function
q_earth_mse_tile = get_earth_pos(obstime_mjd).transpose()

q_earth_mse_tile.shape

(3, 58448)

In [99]:
# shape JPL positions to match q_ast_mse with three axes (asteroid_num, time_idx, space_dim)
q_ast_jpl_3d = np.zeros((N_ast, N_t, 3))
q_ast_jpl_3d[:, :, 0] = df_ast.X.values.reshape((N_ast, N_t))
q_ast_jpl_3d[:, :, 1] = df_ast.Y.values.reshape((N_ast, N_t))
q_ast_jpl_3d[:, :, 2] = df_ast.Z.values.reshape((N_ast, N_t))
q_ast_jpl_3d.shape

(16, 3653, 3)

In [100]:
# Reshape MSE asteroid data to match shape of DataFrame
q_ast_mse = np.zeros((3, N_row))
v_ast_mse = np.zeros((3, N_row))

In [101]:
# Position
q_ast_mse[0, :] = q_ast_mse_3d[:, :, 0].reshape((-1))
q_ast_mse[1, :] = q_ast_mse_3d[:, :, 1].reshape((-1))
q_ast_mse[2, :] = q_ast_mse_3d[:, :, 2].reshape((-1))

# Velocity
v_ast_mse[0, :] = v_ast_mse_3d[:, :, 0].reshape((-1))
v_ast_mse[1, :] = v_ast_mse_3d[:, :, 1].reshape((-1))
v_ast_mse[2, :] = v_ast_mse_3d[:, :, 2].reshape((-1))

In [102]:
# Compute direction from MSE position and velocity
u_mse = qv2dir(q_body=q_ast_mse*au, v_body=v_ast_mse*au/day, q_earth=q_earth_mse_tile*au)

# difference in directions as a vector
u_diff = u_mse - u_jpl

# norm of difference, converted to arc seconds
u_diff_norm = np.linalg.norm(u_diff, axis=0)
angle_diff = np.rad2deg(u_diff_norm)*3600

# mean error in arc-seconds
mean_error = np.mean(angle_diff)
print(f'mean error: {mean_error:8.3f} arc seconds')

mean error:    3.589 arc seconds


**Conclusion:<br>**
My end to end calculation of astromentric RA and DEC are very close to those of JPL.<br>
In my calculation, I am only taking a single snapshot of planetary positions and velocities, plus orbital elements of the asteroids.  Everything else is done by numerically integrating the system in rebound.
I am computing an astrometric direction u on the unit sphere in the ecliptic frame, and comparing this to a direction from JPL.  The JPL direction u_jpl is computed by applying radec2dir() on the quoted RA and DEC.<br>
Errors are on the order of **3.6 arc seconds**.<br>
I am guessing that one main source for the difference with JPL is that I used heliocentric rather than barycentric coordinates when saving the outputs of the rebound integration.  I plan to switch to barycentric for the asteroid search.  Of course there are also some other differences because these are completely separate calculations.  JPL in particular is using many more massive bodies, and they are accounting for relativistic effects.<br>
Still, the bottom line is that an agreement of only 3.6 arc seconds is very tight and suggests that my methodology is basically sound.

### Astrometric vs. Apparent Coordinates

In [103]:
df_obs

,asteroid_num,mjd,JulianDate,time_key,u_x,u_y,u_z,RA,DEC,RA_apparent,DEC_apparent,delta,delta_dot,light_time
0,1,55197.0,2455197.5,1324728,-0.430702,-0.899812,0.069523,243.215442,-17.105913,243.358581,-17.131844,3.437877,-12.468091,28.591952
1,1,55198.0,2455198.5,1324752,-0.424384,-0.902835,0.069195,243.625145,-17.196033,243.768548,-17.221645,3.430618,-12.668422,28.531584
2,1,55199.0,2455199.5,1324776,-0.418063,-0.905804,0.068867,244.034084,-17.284935,244.177730,-17.310227,3.423244,-12.868277,28.470254
3,1,55200.0,2455200.5,1324800,-0.411741,-0.908720,0.068539,244.442231,-17.372621,244.586099,-17.397589,3.415755,-13.067723,28.407966
4,1,55201.0,2455201.5,1324824,-0.405417,-0.911583,0.068211,244.849560,-17.459094,244.993632,-17.483728,3.408150,-13.266749,28.344720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3648,16,58845.0,2458845.5,1412280,0.867713,-0.496854,-0.014471,332.583400,-12.175911,332.842722,-12.079671,3.017646,17.406609,25.096998
3649,16,58846.0,2458846.5,1412304,0.871109,-0.490873,-0.014611,332.963279,-12.044021,333.222363,-11.947459,3.027660,17.268707,25.180280
3650,16,58847.0,2458847.5,1412328,0.874478,-0.484841,-0.014750,333.344527,-11.910963,333.603364,-11.814086,3.037593,17.128990,25.262894
3651,16,58848.0,2458848.5,1412352,0.877821,-0.478758,-0.014889,333.727108,-11.776749,333.985688,-11.679564,3.047446,16.987592,25.344833


**JPL Definitions of Astrometric & Apparent RA/DEC**

 R.A._______(ICRF)_______DEC =
  Astrometric right ascension and declination of the target center with
respect to the observing site (coordinate origin) in the reference frame of
the planetary ephemeris (ICRF). Compensated for down-leg light-time delay
aberration.

  Units: RA  in decimal degrees (ddd.fffffffff)
         DEC in decimal degrees (sdd.fffffffff)

 
 R.A._______(airless-appar)_______DEC. =
  Airless apparent right ascension and declination of the target center with
respect to an instantaneous reference frame defined by the Earth equator
of-date (z-axis) and meridian containing the Earth equinox of-date (x-axis,
IAU76/80). Compensated for down-leg light-time delay, gravitational deflection
of light, stellar aberration, precession & nutation. Note: equinox (RA origin)
is offset -53 mas from the of-date frame defined by the IAU06/00a P & N system.

  Units: RA  in decimal degrees (ddd.fffffffff)
         DEC in decimal degrees (sdd.fffffffff)

In [104]:
# alias the astrometric RA/DEC so the variable names look consistent
# these are the astrometric RA/DEC
ra_astro = ra_jpl
dec_astro = dec_jpl

# arrays of apparent asteroid angles from JPL
ra_appar = df_obs.RA_apparent.values * deg
dec_appar = df_obs.DEC_apparent.values * deg

# Compute difference in angles
diff_app = radec_diff('Astrometric', 'Apparent', ra1=ra_astro, dec1=dec_astro, ra2=ra_appar, dec2=dec_appar, 
                     obstime_mjd=obstime_mjd, verbose=False)
diff_mean = np.mean(diff_app)
diff_median = np.median(diff_app)
diff_max = np.max(diff_app)

# Report results
print(f'Mean Angle Difference: JPL astrometric vs. JPL apparent')
print(f'Mean  : {diff_mean:5.0f} seconds ({(diff_mean/3600):0.3f} degrees)')
print(f'Median: {diff_median:5.0f} seconds ({(diff_median/3600):0.3f} degrees)')
print(f'Max   : {diff_max:5.0f} seconds ({(diff_max/3600):0.3f} degrees)')

Mean Angle Difference: JPL astrometric vs. JPL apparent
Mean  :   743 seconds (0.206 degrees)
Median:   743 seconds (0.206 degrees)
Max   :  1005 seconds (0.279 degrees)


**Conclusion**<br>
The difference between astrometric and apparent RA / DEC is really important!<br>
It's much more important than some of the other effects considered.<br>
It introduces errors on the order of **0.21 degrees / 743 arc seconds**<br>
We need to figure out the quotation basis of the ZTF data!<br>
Francisco from Alerce says he believes ZTF data "must be" astrometric.  Hopefully this is correct!

### Estimate Importance of Including Observatory Location

In [105]:
# Alias old dataframe to df_geocenter
df_geocenter = df_obs

In [106]:
# Load a single data file with Ceres viewed daily from Palomar
# Times will match against df_geocenter exactly

df_pal_daily = pd.read_csv(f'../data/jpl/testing/daily/observer-asteroid-001-palomar.txt', index_col=False)
df_pal_daily = add_cols_ast_observation(df_pal_daily, asteroid_num=1)

In [107]:
df_pal_daily

,asteroid_num,mjd,JulianDate,time_key,u_x,u_y,u_z,RA,DEC,RA_apparent,DEC_apparent,delta,delta_dot,light_time
0,1,55197.0,2455197.5,1324728,-0.430710,-0.899809,0.069515,243.214830,-17.106252,243.357955,-17.132205,3.437890,-12.102732,28.592060
1,1,55198.0,2455198.5,1324752,-0.424392,-0.902831,0.069187,243.624533,-17.196371,243.767921,-17.222005,3.430632,-12.303926,28.531695
2,1,55199.0,2455199.5,1324776,-0.418072,-0.905800,0.068859,244.033471,-17.285272,244.177102,-17.310586,3.423258,-12.504680,28.470368
3,1,55200.0,2455200.5,1324800,-0.411749,-0.908717,0.068531,244.441618,-17.372957,244.585470,-17.397946,3.415769,-12.705061,28.408083
4,1,55201.0,2455201.5,1324824,-0.405426,-0.911580,0.068203,244.848947,-17.459428,244.993002,-17.484084,3.408165,-12.905058,28.344840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3648,1,58845.0,2458845.5,1412280,0.275661,-0.958816,-0.068431,287.919382,-26.370823,288.216441,-26.337182,3.872548,5.527480,32.206999
3649,1,58846.0,2458846.5,1412304,0.282323,-0.956850,-0.068795,288.362991,-26.342168,288.659916,-26.307744,3.875509,5.270949,32.231629
3650,1,58847.0,2458847.5,1412328,0.288973,-0.954836,-0.069160,288.806563,-26.312394,289.103336,-26.277190,3.878321,5.014164,32.255015
3651,1,58848.0,2458848.5,1412352,0.295611,-0.952775,-0.069527,289.250069,-26.281505,289.546673,-26.245524,3.880984,4.757228,32.277158


In [108]:
# alias old df_obs to df_geocenter
df_geocenter = df_obs

# only rows in df_geocenter for ceres
mask = df_geocenter.asteroid_num == 1

# extract RA and DEC for geocenter
ra_geo = df_geocenter[mask].RA.values * deg
dec_geo = df_geocenter[mask].DEC.values * deg

# extract RA and DEC for palomar
ra_pal = df_pal_daily.RA.values * deg
dec_pal = df_pal_daily.DEC.values * deg

In [109]:
# report the angle difference from accounting the observatory
diff_topos = radec_diff('Geocenter', 'Palomar', ra1=ra_geo, dec1=dec_geo, ra2=ra_pal, dec2=dec_pal, 
                         obstime_mjd=obstime_mjd, verbose=True)

Angle Difference: Palomar vs. Geocenter
Mean  :   0.000707 deg (   2.547 seconds)
Median:   0.000645 deg (   2.323 seconds)
Max   :   0.001519 deg (   5.468 seconds)


**Conclusion**<br>
Ignoring the observatory location would introduce an error of **2.55 arc seconds**<br>
This effect is important enough that we should certainly try to model it.

### Incorporate Observatory Location (topos)

In [110]:
def add_cols_ast_obs_site(df, asteroid_num: int):
    """Add columns to a DataFrame with observation from a real site on earth; new columns for asteroid number, mjd, and direction"""
    # Add column for asteroid_num
    df['asteroid_num'] = asteroid_num
    # Add column for mjd
    df['mjd'] = (df['JulianDate'] - 2400000.5)
    # Add integer column for the date/time
    df['time_key'] = np.int32(np.round(df['mjd']*24))
    # Add columns for the direction in ecliptic
    u = radec2dir(ra=df.RA.values * deg, dec=df.DEC.values * deg, obstime_mjd = df['mjd'].values)
    df['u_x'] = u[0]
    df['u_y'] = u[1]
    df['u_z'] = u[2]
    # Compute flag indicating whether observation is possible (no direct sunlight; allow twilight and moonlight)
    df['CanObserve'] = (df['SunIsPresent'] != '*')
    # Filter to only rows where observation is possible
    mask = df['CanObserve'] == True
    df = df[mask]
    # Order columns
    columns = ['asteroid_num', 'mjd', 'JulianDate', 'time_key',
               'u_x', 'u_y', 'u_z', 'RA', 'DEC', 
               'RA_apparent', 'DEC_apparent',
               'delta', 'delta_dot', 'light_time',
               # 'SunIsPresent', 'MoonIsPresent',
              ]
    df = df[columns]
    return df

In [113]:
# Load the JPL position of Earth at 3 hour intervals as CSV
df_earth_3h = pd.read_csv('../data/jpl/testing/hourly/vectors-earth.txt', index_col=False)
# Add columns for date /time
df_earth_3h = add_col_planet(df_earth_3h)

# Load the JPL Position of asteroids at 3 hour intervals as CSV
df_ast_3h = pd.read_csv('../data/jpl/testing/hourly/vectors-earth.txt', index_col=False)

# Display the earth dataframe
df_earth_3h

,mjd,JulianDate,time_key,X,Y,Z,VX,VY,VZ,LT,RG,RR
0,55197.000,2455197.500,1324728,-0.179765,0.970347,-0.000017,-0.017202,-0.003148,8.961125e-07,0.005700,0.986858,0.000038
1,55197.125,2455197.625,1324731,-0.181915,0.969951,-0.000017,-0.017195,-0.003186,8.995828e-07,0.005700,0.986863,0.000039
2,55197.250,2455197.750,1324734,-0.184064,0.969551,-0.000017,-0.017188,-0.003223,9.023349e-07,0.005700,0.986868,0.000039
3,55197.375,2455197.875,1324737,-0.186212,0.969145,-0.000017,-0.017181,-0.003260,9.043645e-07,0.005700,0.986873,0.000040
4,55197.500,2455198.000,1324740,-0.188359,0.968736,-0.000017,-0.017174,-0.003298,9.056684e-07,0.005700,0.986878,0.000041
...,...,...,...,...,...,...,...,...,...,...,...,...
29212,58848.500,2458849.000,1412364,-0.161514,0.978014,-0.000019,-0.017273,-0.002832,7.120095e-07,0.005725,0.991261,0.000020
29213,58848.625,2458849.125,1412367,-0.163673,0.977658,-0.000018,-0.017267,-0.002870,6.982338e-07,0.005725,0.991264,0.000020
29214,58848.750,2458849.250,1412370,-0.165831,0.977297,-0.000018,-0.017261,-0.002908,6.842707e-07,0.005725,0.991266,0.000021
29215,58848.875,2458849.375,1412373,-0.167988,0.976931,-0.000018,-0.017254,-0.002946,6.701300e-07,0.005725,0.991269,0.000021


In [116]:
# List of dataframes; one per asteroid
df_ast_list_3h = []

# Load the JPL position of asteroids one at a time
for j in range(1, 17):
    # print(j)
    df_ast_j = pd.read_csv(f'../data/jpl/testing/hourly/vectors-asteroid-{j:03d}.txt', index_col=False)
    # Add columns for the asteroid_num and mjd
    df_ast_j = add_cols_ast_vector(df=df_ast_j, asteroid_num=j)
    # Add this to list of frames
    df_ast_list_3h.append(df_ast_j)

# Concatenate dataframes
df_ast_3h = pd.concat(df_ast_list_3h)

In [117]:
df_ast_3h

,asteroid_num,mjd,JulianDate,time_key,X,Y,Z,VX,VY,VZ,LT,RG,RR
0,1,55197.000,2455197.500,1324728,-1.660333,-2.123236,0.238962,0.007615,-0.007150,-0.001627,0.015628,2.705909,0.000794
1,1,55197.125,2455197.625,1324731,-1.659381,-2.124130,0.238759,0.007618,-0.007146,-0.001627,0.015629,2.706008,0.000794
2,1,55197.250,2455197.750,1324734,-1.658429,-2.125023,0.238555,0.007622,-0.007142,-0.001628,0.015629,2.706107,0.000794
3,1,55197.375,2455197.875,1324737,-1.657476,-2.125915,0.238352,0.007625,-0.007138,-0.001628,0.015630,2.706206,0.000794
4,1,55197.500,2455198.000,1324740,-1.656522,-2.126807,0.238148,0.007628,-0.007134,-0.001628,0.015630,2.706306,0.000794
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29212,16,58848.500,2458849.000,1412364,2.522975,-0.473790,-0.045683,0.001437,0.011240,-0.000566,0.014829,2.567482,-0.000652
29213,16,58848.625,2458849.125,1412367,2.523154,-0.472385,-0.045754,0.001431,0.011242,-0.000566,0.014828,2.567401,-0.000652
29214,16,58848.750,2458849.250,1412370,2.523332,-0.470980,-0.045825,0.001426,0.011243,-0.000566,0.014828,2.567319,-0.000651
29215,16,58848.875,2458849.375,1412373,2.523510,-0.469574,-0.045895,0.001420,0.011244,-0.000565,0.014827,2.567238,-0.000650


In [118]:
# Load palomar observations at 3 hour intervals, filtered to eliminate daylight

# List of dataframes; one per asteroid
df_obs_list = []

# Load the JPL observations of asteroids one at a time
for j in range(1, 17):
    # print(j)
    df_obs_j = pd.read_csv(f'../data/jpl/testing/hourly/observer-asteroid-{j:03d}-palomar.txt', index_col=False)
    # Add columns for the asteroid_num and mjd
    df_obs_j = add_cols_ast_obs_site(df=df_obs_j, asteroid_num=j)
    # Add this to list of frames
    df_obs_list.append(df_obs_j)

# Concatenate dataframes
df_palomar = pd.concat(df_obs_list)

In [119]:
df_palomar

,asteroid_num,mjd,JulianDate,time_key,u_x,u_y,u_z,RA,DEC,RA_apparent,DEC_apparent,delta,delta_dot,light_time
1,1,55197.125,2455197.625,1324731,-0.429917,-0.900191,0.069477,243.266342,-17.117471,243.409452,-17.143375,3.437011,-12.282233,28.584749
2,1,55197.250,2455197.750,1324734,-0.429121,-0.900574,0.069438,243.318051,-17.128756,243.461181,-17.154603,3.436114,-12.585595,28.577286
3,1,55197.375,2455197.875,1324737,-0.428325,-0.900956,0.069397,243.369682,-17.140123,243.512878,-17.165916,3.435195,-12.849298,28.569644
4,1,55197.500,2455198.000,1324740,-0.427533,-0.901335,0.069354,243.420999,-17.151530,243.564287,-17.177279,3.434263,-12.933136,28.561893
9,1,55198.125,2455198.625,1324755,-0.423599,-0.903207,0.069149,243.675956,-17.207437,243.819326,-17.233022,3.429738,-12.485681,28.524262
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29204,16,58847.500,2458848.000,1412340,0.876153,-0.481806,-0.014825,333.535748,-11.844294,333.794383,-11.747260,3.042569,17.005626,25.304275
29209,16,58848.125,2458848.625,1412355,0.878231,-0.478004,-0.014911,333.774472,-11.760401,334.033066,-11.663191,3.048655,17.275036,25.354893
29210,16,58848.250,2458848.750,1412358,0.878646,-0.477241,-0.014926,333.822279,-11.743425,334.080784,-11.646180,3.049906,17.328372,25.365293
29211,16,58848.375,2458848.875,1412361,0.879063,-0.476471,-0.014943,333.870504,-11.726444,334.128926,-11.629155,3.051152,17.160449,25.375657


In [120]:
# Earth location of the Palomar observatory
geoloc_palomar_auto = EarthLocation.of_site('Palomar')
print(f'geolocation of Palomar observatory:')
print(f'geoloc_palomar = {geoloc_palomar_auto}')
print(f'geoloc_palomar geodetic = {geoloc_palomar_auto.geodetic}')

geolocation of Palomar observatory:
geoloc_palomar = (-2410346.78217658, -4758666.82504051, 3487942.97502457) m
geoloc_palomar geodetic = GeodeticLocation(lon=<Longitude -116.863 deg>, lat=<Latitude 33.356 deg>, height=<Quantity 1706. m>)


In [121]:
# For some reason, the 'nice' way to do this below leads to a bug in constructor to GCRS
# geoloc_palomar = EarthLocation(-2410346.8, -4758666.8, 3487943, unit='m')

# Workaround: extract the geolocation of the Palomar observatory using EarthLocation.of_site
geoloc_palomar = [-2410346.78217658, -4758666.82504051, 3487942.97502457] * meter

In [122]:
obsgeoloc = geoloc_palomar
obsgeoloc

<Quantity [-2410346.78217658, -4758666.82504051,  3487942.97502457] m>

**Make sure calculations from geocenter still work**

In [123]:
# the direction according to JPL; convert RA/DEC to direction in ecliptic plane
u_jpl = radec2dir(ra=ra_jpl, dec=dec_jpl, obstime_mjd=obstime_mjd)

# calculate direction with JPL data
u_mse = qv2dir(q_body=q_ast_jpl, v_body=v_ast_jpl, q_earth=q_earth_jpl_tile)

# difference in directions as a vector
u_diff = u_mse - u_jpl

# norm of difference, converted to arc seconds
u_diff_norm = np.linalg.norm(u_diff, axis=0)
angle_diff = np.rad2deg(u_diff_norm)*3600

# mean error in arc-seconds
mean_error = np.mean(angle_diff)
print(f'Observation of Asteroids: MSE qv2dir() on JPL position vs. JPL direction')
print(f'mean error: {mean_error:8.3f} arc seconds')

Observation of Asteroids: MSE qv2dir() on JPL position vs. JPL direction
mean error:    0.873 arc seconds


**Observations from Palomar According to JPL; Augment wit Interpolated Earth & Asteroid Position**

In [124]:
def obs_add_interp_qv(df_obs: pd.DataFrame, 
                      df_ast: pd.DataFrame,
                      df_earth: pd.DataFrame,) -> None:
    """Add interpolated position and velocity to a DataFrame of asteroid observations"""
    
    # Get list of distinct asteroid numbers
    asteroid_nums = list(set(df_obs.asteroid_num.values))

    # Add new columns for position and velocity
    cols = ['ast_x', 'ast_y', 'ast_z', 'ast_vx', 'ast_vy', 'ast_vz']
    for col in cols:
        df_obs[col] = 0.0
        
    # Interpolator for earth position
    interp_t = df_earth.mjd.values
    interp_q = df_earth[['X', 'Y', 'Z']].values
    interp_earth = CubicSpline(x=interp_t, y=interp_q)
    
    # Set interpolated position of earth
    earth_cols = ['earth_x', 'earth_y', 'earth_z']
    earth_q = interp_earth(df_obs.mjd.values)
    for k, col in enumerate(earth_cols):
        df_obs[col] = earth_q[:, k]

    # Add interpolated position and velocity to df_palomar
    for asteroid_num in asteroid_nums:

        # Masks for this asteroid on the vector and observation dataframes
        mask_vec = (df_ast.asteroid_num == asteroid_num)
        mask_obs = (df_obs.asteroid_num == asteroid_num)

        # Build an interpolator for the asteroid position and velocity
        interp_t = df_ast[mask_vec].mjd.values
        interp_qv = df_ast[mask_vec][['X', 'Y', 'Z', 'VX', 'VY', 'VZ']].values
        interp_ast = CubicSpline(x=interp_t, y=interp_qv)

        # The times to be interpolated
        obs_t = df_obs[mask_obs].mjd.values

        # Evaluate the interpolated position / velocity at the observation times
        obs_qv = interp_ast(obs_t)

        # Assign interpolated qv to the df_obs dataframe on the mask
        for k, col in enumerate(cols):
            df_obs.loc[mask_obs, col] = obs_qv[:, k]

In [125]:
# Add interpolated position and velocity variables according to JPL
obs_add_interp_qv(df_ast=df_ast_3h, df_earth=df_earth_3h, df_obs=df_palomar)

In [126]:
# Review the augmented observation table
df_palomar

,asteroid_num,mjd,JulianDate,time_key,u_x,u_y,u_z,RA,DEC,RA_apparent,...,light_time,ast_x,ast_y,ast_z,ast_vx,ast_vy,ast_vz,earth_x,earth_y,earth_z
1,1,55197.125,2455197.625,1324731,-0.429917,-0.900191,0.069477,243.266342,-17.117471,243.409452,...,28.584749,-1.659381,-2.124130,0.238759,0.007618,-0.007146,-0.001627,-0.181915,0.969951,-0.000017
2,1,55197.250,2455197.750,1324734,-0.429121,-0.900574,0.069438,243.318051,-17.128756,243.461181,...,28.577286,-1.658429,-2.125023,0.238555,0.007622,-0.007142,-0.001628,-0.184064,0.969551,-0.000017
3,1,55197.375,2455197.875,1324737,-0.428325,-0.900956,0.069397,243.369682,-17.140123,243.512878,...,28.569644,-1.657476,-2.125915,0.238352,0.007625,-0.007138,-0.001628,-0.186212,0.969145,-0.000017
4,1,55197.500,2455198.000,1324740,-0.427533,-0.901335,0.069354,243.420999,-17.151530,243.564287,...,28.561893,-1.656522,-2.126807,0.238148,0.007628,-0.007134,-0.001628,-0.188359,0.968736,-0.000017
9,1,55198.125,2455198.625,1324755,-0.423599,-0.903207,0.069149,243.675956,-17.207437,243.819326,...,28.524262,-1.651750,-2.131260,0.237130,0.007643,-0.007114,-0.001631,-0.199081,0.966616,-0.000016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29204,16,58847.500,2458848.000,1412340,0.876153,-0.481806,-0.014825,333.535748,-11.844294,333.794383,...,25.304275,2.521516,-0.485026,-0.045117,0.001481,0.011232,-0.000567,-0.144216,0.980695,-0.000019
29209,16,58848.125,2458848.625,1412355,0.878231,-0.478004,-0.014911,333.774472,-11.760401,334.033066,...,25.354893,2.522433,-0.478005,-0.045471,0.001453,0.011237,-0.000566,-0.155033,0.979055,-0.000019
29210,16,58848.250,2458848.750,1412358,0.878646,-0.477241,-0.014926,333.822279,-11.743425,334.080784,...,25.365293,2.522614,-0.476600,-0.045542,0.001448,0.011238,-0.000566,-0.157194,0.978713,-0.000019
29211,16,58848.375,2458848.875,1412361,0.879063,-0.476471,-0.014943,333.870504,-11.726444,334.128926,...,25.375657,2.522795,-0.475195,-0.045612,0.001442,0.011239,-0.000566,-0.159354,0.978366,-0.000019


In [127]:
from typing import Optional
light_speed = astropy.constants.c.to(au / minute)

In [128]:
zero_deg = 0.0 * deg
zero_meter = 0.0 * meter
zero_au = 0.0 * au

# The observation time
obstime = astropy.time.Time(58600.0, format='mjd')

In [129]:
def qv2dir(q_body: np.ndarray, v_body: np.ndarray, q_earth: np.ndarray, 
           obstime_mjd: Optional[np.ndarray] = None, obsgeoloc: Optional[np.ndarray] = None) -> np.ndarray:
    """
    Compute the direction of displacement from earth to a space body as a unit displacement vector
    u = (ux, uy, uz) in the ecliptic plane.
    INPUTS:
        q_body: position of this body in ecliptic coordinate frame; passed with units (default AU)
        v_body: velocity of this body in ecliptic coordinate frame; passed with units (default AU / day)
        q_earth: position of earth in ecliptic coordinate frame; passed with units (default AU)
        obstime_mjd: observation time as a modified julian date; only required if passing obsgeoloc 
        obsgeoloc: geolocation of the observatory as vector [X, Y, Z] in meters using ITRS
    RETURNS:
        u: An array [ux, uy, uz] on the unit sphere in the ecliptic frame
    EXAMPLE:
        u = qv2dir(q_body=np.array([-0.328365, 1.570624, 0.040733])*au, 
                   v_body=np.array([-0.013177, -0.001673, 0.000288])*au/day,
                   q_earth=np.array([-0.813785, -0.586761, -0.000003])*au,
                   obsgeoloc=[-2410346.78217658, -4758666.82504051, 3487942.97502457] * meter)
    """
    # compute the correction due to the observatory of obstime_mjd and geoloc are passed
    # dq_obs is the displacement from geocenter to the observatory
    if (obstime_mjd is not None and obsgeoloc is not None):
        # the observation times as astropy time objects
        obstime = astropy.time.Time(obstime_mjd, format='mjd')
        # the displacement from the geocenter to the observatory in the ICRS frame
        x, y, z = obsgeoloc
        obs_icrs = SkyCoord(x=x, y=y, z=z, obstime=obstime, frame=ICRS, representation_type='cartesian')
        # displacement from geocenter to observatory in the BME frame
        obs_bme = obs_icrs.transform_to(BarycentricMeanEcliptic)
        dq_topos = obs_bme.cartesian.xyz.to(au).reshape((-1, 1))
    else:
        # default is to use geocenter if obstime and geoloc are note passed
        dq_topos = np.zeros((3,1)) * au
        
    # position of the observer in space
    q_obs = q_earth + dq_topos

    # displacement from observer on earth to body; in AU
    q_rel = q_body - q_obs

    # distance; in AU
    r = np.linalg.norm(q_rel, axis=0) * au
    
    # light time in minutes
    light_time = (r / light_speed)
    
    # adjusted relative position, accounting for light time
    dq_lt = light_time * v_body.to(au/minute)     # convert velocity to au / minute b/c time is in minutes
    q_rel_lt = q_rel - dq_lt
    
    # adjusted direction
    r_lt = np.linalg.norm(q_rel_lt, axis=0) * au
    u = q_rel_lt / r_lt
    return u.value

In [130]:
# extract the position and velocity according to JPL; same alignment as Palomar obs data
q_ast_pal_jpl = df_palomar[['ast_x', 'ast_y', 'ast_z']].values.transpose() * au
v_ast_pal_jpl = df_palomar[['ast_vx', 'ast_vy', 'ast_vz']].values.transpose() * au / day
q_earth_pal_jpl = df_palomar[['earth_x', 'earth_y', 'earth_z']].values.transpose() * au

# extract the RA and DEC from Palomar according to JPL
ra_pal_jpl = df_palomar.RA.values * deg
dec_pal_jpl = df_palomar.DEC.values * deg
obstime_pal_mjd = df_palomar.mjd.values

In [131]:
# the direction according to JPL; convert RA/DEC to direction in ecliptic plane
u_pal_jpl = radec2dir(ra=ra_pal_jpl, dec=dec_pal_jpl, obstime_mjd=obstime_pal_mjd)

# calculate direction with JPL data
u_pal_mse = qv2dir(q_body=q_ast_pal_jpl, v_body=v_ast_pal_jpl, q_earth=q_earth_pal_jpl, obstime_mjd=obstime_pal_mjd, obsgeoloc=geoloc_palomar)

# difference in directions as a vector
u_diff = u_pal_mse - u_pal_jpl

# norm of difference, converted to arc seconds
u_diff_norm = np.linalg.norm(u_diff, axis=0)
angle_diff = np.rad2deg(u_diff_norm)*3600

# mean error in arc-seconds
mean_error = np.mean(angle_diff)
print(f'Observation of Asteroids: MSE qv2dir() on JPL position vs. JPL direction')
print(f'mean error: {mean_error:8.3f} arc seconds')

Observation of Asteroids: MSE qv2dir() on JPL position vs. JPL direction
mean error:    2.879 arc seconds


In [132]:
u_pal_mse = qv2dir(q_body=q_ast_pal_jpl, v_body=v_ast_pal_jpl, q_earth=q_earth_pal_jpl, obstime_mjd=obstime_pal_mjd, obsgeoloc=geoloc_palomar)
u_geo_mse = qv2dir(q_body=q_ast_pal_jpl, v_body=v_ast_pal_jpl, q_earth=q_earth_pal_jpl, obstime_mjd=obstime_pal_mjd, obsgeoloc=np.zeros(3)*meter)

In [133]:
# difference between MSE palomar and MSE geocenter; expect this to be around 3 arc seconds
np.rad2deg(np.mean(np.linalg.norm(u_pal_mse - u_geo_mse, axis=0)))*3600

3.146769631452609

In [135]:
# difference between JPL palomar and MSE from geocenter; expect error to be larger by about 3 arc seconds
np.rad2deg(np.mean(np.linalg.norm(u_geo_mse - u_pal_jpl, axis=0)))*3600

2.928311221290333

In [134]:
# difference between MSE palomar and JPL palomar; want this to be very small
np.rad2deg(np.mean(np.linalg.norm(u_pal_mse - u_pal_jpl, axis=0)))*3600

2.8793010708419122